# Recession Resilience

This project examined whether university towns have their mean housing prices less effected by recessions. The raw date are stored in three different files. 

* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in quarterly intervals is in the file ```gdplev.xls```. 

In [187]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [188]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [191]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. 
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.'''

    university_towns = []
    with open('university_towns.txt','r') as file:
        for line in file:
            if line.strip().endswith('[edit]'):
                state = line.split('[')[0]
            else:
                university_towns.append([state, line.split(' (')[0]])
    university_towns_df = pd.DataFrame(university_towns, columns = ["State", "RegionName"])
    university_towns_df['UniversityTowns'] = 1
    return university_towns_df
get_list_of_university_towns().head()

,State,RegionName,UniversityTowns
0,Alabama,Auburn,1
1,Alabama,Florence,1
2,Alabama,Jacksonville,1
3,Alabama,Livingston,1
4,Alabama,Montevallo,1


In [192]:
# Load the GDP data and calculate the GDP change from quarter to quarter
gdp = pd.read_excel('gdplev.xls',skiprows = 5)[214:280][['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']] \
    .rename(columns = {'Unnamed: 4':'Quarter', 'GDP in billions of chained 2009 dollars.1':'GDP'}) \
    .set_index('Quarter')
gdp['Change'] = gdp['GDP'].diff()
gdp.head()

,GDP,Change
Quarter,,
2000q1,12359.1,NaN
2000q2,12592.5,233.4
2000q3,12607.7,15.2
2000q4,12679.3,71.6
2001q1,12643.3,-36.0


Definitions:
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.

In [83]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time '''
    for i in range(len(gdp)-1):
        if (gdp.iloc[i]['Change'] < 0) & (gdp.iloc[i+1]['Change'] < 0):
            return gdp.iloc[i].name
get_recession_start()

'2008q3'

In [84]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time '''
    
    for i in range(1,len(gdp)):
        if (gdp.iloc[i].name > get_recession_start()) \
            & (gdp.iloc[i-1]['Change'] > 0) \
            & (gdp.iloc[i]['Change'] > 0):
            return gdp.iloc[i].name     
get_recession_end()

'2009q4'

In [93]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time '''
    return gdp['GDP'].loc[get_recession_start():get_recession_end()].idxmin()
get_recession_bottom()

'2009q2'

In [200]:
# Load the housing data 
house = pd.read_csv('City_Zhvi_AllHomes.csv') \
        .drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis = 1) 
house['State'] = house['State'].apply(lambda x: states[x])
house = house.set_index(['State','RegionName']) \
            .loc[:,'2000-01':] 
house.columns

Index(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06',
       '2000-07', '2000-08', '2000-09', '2000-10',
       ...
       '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04',
       '2016-05', '2016-06', '2016-07', '2016-08'],
      dtype='object', length=200)

In [199]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. '''
    housing_data_to_quarters = house.groupby(pd.PeriodIndex(house.columns,freq = 'q'), axis =1).mean()
    return housing_data_to_quarters
convert_housing_data_to_quarters().columns

PeriodIndex(['2000Q1', '2000Q2', '2000Q3', '2000Q4', '2001Q1', '2001Q2',
             '2001Q3', '2001Q4', '2002Q1', '2002Q2', '2002Q3', '2002Q4',
             '2003Q1', '2003Q2', '2003Q3', '2003Q4', '2004Q1', '2004Q2',
             '2004Q3', '2004Q4', '2005Q1', '2005Q2', '2005Q3', '2005Q4',
             '2006Q1', '2006Q2', '2006Q3', '2006Q4', '2007Q1', '2007Q2',
             '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3', '2008Q4',
             '2009Q1', '2009Q2', '2009Q3', '2009Q4', '2010Q1', '2010Q2',
             '2010Q3', '2010Q4', '2011Q1', '2011Q2', '2011Q3', '2011Q4',
             '2012Q1', '2012Q2', '2012Q3', '2012Q4', '2013Q1', '2013Q2',
             '2013Q3', '2013Q4', '2014Q1', '2014Q2', '2014Q3', '2014Q4',
             '2015Q1', '2015Q2', '2015Q3', '2015Q4', '2016Q1', '2016Q2',
             '2016Q3'],
            dtype='period[Q-DEC]', freq='Q-DEC')

In [190]:
def run_ttest():
    '''Create new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then run a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The value for better is 
    "university town" or "non-university town" depending on which has a lower 
    mean price ratio.'''
    start = get_recession_start().upper()
    end = get_recession_bottom().upper()
    housing_data = convert_housing_data_to_quarters()
    university_towns_df = get_list_of_university_towns()
    
    housing_price_change = (housing_data[end]-housing_data[start])/housing_data[start]
    housing_price_change_df = pd.DataFrame(housing_price_change, columns = ['HousingPriceChange']) \
                                .reset_index() \
                                .dropna()

    merge = pd.merge(housing_price_change_df, university_towns_df, how = 'left', left_on = ['State','RegionName'], right_on = ['State','RegionName']) 
    university_town = merge[merge['UniversityTowns'] == 1.0]['HousingPriceChange']
    non_university_town = merge[merge['UniversityTowns'] != 1.0]['HousingPriceChange']
    
    t,p = ttest_ind(university_town, non_university_town)
    better = 'university town' if university_town.mean() > non_university_town.mean() else 'non-university town'                
    return (p<0.01, p, better)
run_ttest()

(True, 0.008975587540712617, 'university town')